# 제조 AI 실습 과정 - Session 1 (0-2시간)
## 제조 데이터 아키텍처 & 과제 정의

### 학습 목표
- OT/IT 구조(PLC/SCADA/MES)와 데이터 맵 이해
- 품질/설비/공정 과제 발굴 및 KPI/ROI 정의 방법 습득
- 데이터 거버넌스 보안/접근권한 기본 원칙 학습

---

## 1. 환경 설정 및 필요 라이브러리 설치

한글 깨짐 방지

Step 1: 실행

In [ ]:
!apt-get install -y fonts-nanum*
!rm -rf /root/.cache/matplotlib/* # 폰트 캐시 재설정
print("=" * 50)
print("✅ 폰트 설치 완료!")
print("⚠️ 이제 다음을 수행하세요:")
print("1. 상단 메뉴: 런타임 → 런타임 다시 시작")
print("2. 재시작 후 Step 2 코드 실행")
print("=" * 50)

Step 2: 실행

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
#
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
plt.rcParams['font.family'] = font_name

# 확인
plt.plot([1,2,3], [1,2,3])
plt.title('한글 테스트 성공! 🎉')
plt.xlabel('한글 X축')
plt.ylabel('한글 Y축')
plt.show()

In [ ]:
# 기본 라이브러리 설치
!pip install pandas numpy matplotlib seaborn plotly -q
!pip install opcua paho-mqtt pymodbus -q  # 산업 프로토콜 라이브러리
!pip install faker -q  # 시뮬레이션 데이터 생성용

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings('ignore')



print("✅ 라이브러리 로드 완료")

## 2. OT/IT 구조 이해하기

### 2.1 제조 데이터 계층 구조 (Purdue Model)

In [ ]:
# 제조 데이터 계층 구조 시뮬레이션
class ManufacturingArchitecture:
    def __init__(self):
        self.layers = {
            'Level 0': {'name': 'Physical Process', 'devices': ['센서', '액추에이터', '밸브', '모터']},
            'Level 1': {'name': 'Basic Control', 'devices': ['PLC', 'DCS', 'RTU', 'Smart Sensors']},
            'Level 2': {'name': 'Supervisory Control', 'devices': ['SCADA', 'HMI', 'Engineering Workstation']},
            'Level 3': {'name': 'Operations Management', 'devices': ['MES', 'LIMS', 'WMS', 'Historian']},
            'Level 4': {'name': 'Business Planning', 'devices': ['ERP', 'SCM', 'CRM']}
        }

    def visualize_architecture(self):
        fig = go.Figure()

        y_positions = [0, 1, 2, 3, 4]
        colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57']

        for i, (level, info) in enumerate(self.layers.items()):
            # 레벨 박스 추가
            fig.add_trace(go.Scatter(
                x=[0.5, 3.5, 3.5, 0.5, 0.5],
                y=[y_positions[i]-0.3, y_positions[i]-0.3, y_positions[i]+0.3, y_positions[i]+0.3, y_positions[i]-0.3],
                fill='toself',
                fillcolor=colors[i],
                opacity=0.3,
                line=dict(color=colors[i], width=2),
                name=level,
                text=f"{level}: {info['name']}<br>Devices: {', '.join(info['devices'])}",
                hovertemplate='%{text}<extra></extra>'
            ))

            # 레벨 텍스트 추가
            fig.add_annotation(
                x=2, y=y_positions[i],
                text=f"<b>{level}: {info['name']}</b><br>{', '.join(info['devices'][:2])}...",
                showarrow=False,
                font=dict(size=10)
            )

        fig.update_layout(
            title="제조 데이터 아키텍처 (Purdue Model)",
            showlegend=True,
            height=600,
            xaxis=dict(visible=False, range=[0, 4]),
            yaxis=dict(visible=False, range=[-0.5, 4.5]),
            hovermode='closest'
        )

        return fig

# 아키텍처 시각화
arch = ManufacturingArchitecture()
fig = arch.visualize_architecture()
fig.show()

### 2.2 PLC 데이터 시뮬레이션

In [ ]:
# PLC 태그 데이터 시뮬레이션
class PLCSimulator:
    def __init__(self):
        self.tags = {
            'Temperature_Zone1': {'type': 'REAL', 'value': 0, 'unit': '°C', 'range': [20, 80]},
            'Pressure_Main': {'type': 'REAL', 'value': 0, 'unit': 'bar', 'range': [1, 10]},
            'Motor_Speed': {'type': 'INT', 'value': 0, 'unit': 'RPM', 'range': [0, 3000]},
            'Valve_Status': {'type': 'BOOL', 'value': False, 'unit': '', 'range': [0, 1]},
            'Product_Count': {'type': 'DINT', 'value': 0, 'unit': 'pcs', 'range': [0, 10000]},
            'Alarm_Active': {'type': 'BOOL', 'value': False, 'unit': '', 'range': [0, 1]}
        }

    def generate_realtime_data(self, duration_seconds=60, sampling_rate=1):
        """실시간 PLC 데이터 생성"""
        timestamps = []
        data = {tag: [] for tag in self.tags.keys()}

        for t in range(0, duration_seconds, sampling_rate):
            timestamps.append(datetime.now() + timedelta(seconds=t))

            # 각 태그별 데이터 생성
            for tag, config in self.tags.items():
                if config['type'] == 'BOOL':
                    value = np.random.choice([True, False])
                elif config['type'] in ['REAL', 'INT', 'DINT']:
                    # 노이즈가 포함된 신호 생성
                    base_value = np.mean(config['range'])
                    noise = np.random.normal(0, (config['range'][1] - config['range'][0]) * 0.1)
                    value = np.clip(base_value + noise, config['range'][0], config['range'][1])

                    if config['type'] in ['INT', 'DINT']:
                        value = int(value)

                data[tag].append(value)

        df = pd.DataFrame(data, index=timestamps)
        return df

# PLC 데이터 생성 및 시각화
plc = PLCSimulator()
plc_data = plc.generate_realtime_data(duration_seconds=300, sampling_rate=5)

print("📊 PLC 데이터 샘플:")
print(plc_data.head(10))
print(f"\n✅ 생성된 데이터 포인트: {len(plc_data)} 개")

In [ ]:
# PLC 데이터 시각화
fig, axes = plt.subplots(3, 2, figsize=(15, 10))
fig.suptitle('PLC 실시간 데이터 모니터링', fontsize=16, fontweight='bold')

# 각 태그별 플롯
tags_to_plot = ['Temperature_Zone1', 'Pressure_Main', 'Motor_Speed',
                'Valve_Status', 'Product_Count', 'Alarm_Active']

for idx, (ax, tag) in enumerate(zip(axes.flat, tags_to_plot)):
    ax.plot(plc_data.index, plc_data[tag], linewidth=1.5)
    ax.set_title(f'{tag} ({plc.tags[tag]["unit"]})')
    ax.set_xlabel('시간')
    ax.set_ylabel('값')
    ax.grid(True, alpha=0.3)

    # 경보 임계값 표시 (예시)
    if tag == 'Temperature_Zone1':
        ax.axhline(y=70, color='r', linestyle='--', label='High Alarm')
        ax.axhline(y=30, color='b', linestyle='--', label='Low Alarm')
        ax.legend()

plt.tight_layout()
plt.show()

### 2.3 SCADA 시스템 데이터 통합

In [ ]:
class SCADASystem:
    def __init__(self, plc_data):
        self.plc_data = plc_data
        self.alarm_log = []
        self.kpi_metrics = {}

    def detect_alarms(self, thresholds):
        """알람 감지 및 로깅"""
        for tag, limits in thresholds.items():
            if tag in self.plc_data.columns:
                # High alarm
                high_alarms = self.plc_data[self.plc_data[tag] > limits['high']]
                for timestamp, row in high_alarms.iterrows():
                    self.alarm_log.append({
                        'timestamp': timestamp,
                        'tag': tag,
                        'type': 'HIGH',
                        'value': row[tag],
                        'threshold': limits['high'],
                        'priority': limits.get('priority', 'Medium')
                    })

                # Low alarm
                low_alarms = self.plc_data[self.plc_data[tag] < limits['low']]
                for timestamp, row in low_alarms.iterrows():
                    self.alarm_log.append({
                        'timestamp': timestamp,
                        'tag': tag,
                        'type': 'LOW',
                        'value': row[tag],
                        'threshold': limits['low'],
                        'priority': limits.get('priority', 'Medium')
                    })

        return pd.DataFrame(self.alarm_log)

    def calculate_kpis(self):
        """KPI 계산"""
        # OEE (Overall Equipment Effectiveness) 계산 시뮬레이션
        availability = 0.85 + np.random.uniform(-0.05, 0.05)
        performance = 0.90 + np.random.uniform(-0.05, 0.05)
        quality = 0.98 + np.random.uniform(-0.02, 0.02)

        self.kpi_metrics['OEE'] = availability * performance * quality
        self.kpi_metrics['Availability'] = availability
        self.kpi_metrics['Performance'] = performance
        self.kpi_metrics['Quality'] = quality

        # 추가 KPI
        self.kpi_metrics['MTBF'] = np.random.uniform(150, 200)  # Mean Time Between Failures (hours)
        self.kpi_metrics['MTTR'] = np.random.uniform(2, 5)      # Mean Time To Repair (hours)
        self.kpi_metrics['Throughput'] = plc_data['Product_Count'].diff().mean() * 60  # units/hour

        return self.kpi_metrics

# SCADA 시스템 초기화 및 알람 설정
scada = SCADASystem(plc_data)

# 알람 임계값 설정
alarm_thresholds = {
    'Temperature_Zone1': {'high': 70, 'low': 30, 'priority': 'High'},
    'Pressure_Main': {'high': 8, 'low': 2, 'priority': 'Medium'},
    'Motor_Speed': {'high': 2800, 'low': 100, 'priority': 'Low'}
}

# 알람 감지
alarm_df = scada.detect_alarms(alarm_thresholds)
print(f"🚨 감지된 알람 수: {len(alarm_df)}")
if len(alarm_df) > 0:
    print("\n최근 5개 알람:")
    print(alarm_df.head())

### 2.4 MES 데이터 통합 및 분석

In [ ]:
class MESSystem:
    def __init__(self):
        self.production_orders = self.generate_production_orders()
        self.quality_data = self.generate_quality_data()
        self.inventory_data = self.generate_inventory_data()

    def generate_production_orders(self, n_orders=20):
        """생산 주문 데이터 생성"""
        orders = []
        products = ['Product_A', 'Product_B', 'Product_C', 'Product_D']

        for i in range(n_orders):
            order = {
                'order_id': f'ORD-{i+1000}',
                'product': np.random.choice(products),
                'quantity_planned': np.random.randint(100, 1000),
                'quantity_produced': 0,
                'start_time': datetime.now() - timedelta(hours=np.random.randint(0, 72)),
                'status': np.random.choice(['Planned', 'In Progress', 'Completed', 'On Hold']),
                'priority': np.random.choice(['High', 'Medium', 'Low']),
                'line': f'Line_{np.random.randint(1, 4)}'
            }

            if order['status'] == 'Completed':
                order['quantity_produced'] = order['quantity_planned']
            elif order['status'] == 'In Progress':
                order['quantity_produced'] = np.random.randint(0, order['quantity_planned'])

            orders.append(order)

        return pd.DataFrame(orders)

    def generate_quality_data(self, n_samples=100):
        """품질 검사 데이터 생성"""
        quality_data = []

        for i in range(n_samples):
            sample = {
                'sample_id': f'QC-{i+1000}',
                'timestamp': datetime.now() - timedelta(minutes=np.random.randint(0, 1440)),
                'product': np.random.choice(['Product_A', 'Product_B', 'Product_C', 'Product_D']),
                'dimension_1': np.random.normal(10.0, 0.1),  # 목표: 10.0 ± 0.2
                'dimension_2': np.random.normal(5.0, 0.05),  # 목표: 5.0 ± 0.1
                'weight': np.random.normal(100, 2),          # 목표: 100 ± 5
                'defect_type': np.random.choice(['None', 'Surface', 'Dimension', 'Weight', 'Other'], p=[0.85, 0.05, 0.04, 0.03, 0.03]),
                'inspector': np.random.choice(['Inspector_A', 'Inspector_B', 'Inspector_C'])
            }

            # Pass/Fail 판정
            sample['result'] = 'Pass' if sample['defect_type'] == 'None' else 'Fail'

            quality_data.append(sample)

        return pd.DataFrame(quality_data)

    def generate_inventory_data(self):
        """재고 데이터 생성"""
        materials = ['Raw_Material_1', 'Raw_Material_2', 'Raw_Material_3',
                    'Component_A', 'Component_B', 'Component_C']

        inventory = []
        for material in materials:
            inv = {
                'material': material,
                'current_stock': np.random.randint(100, 5000),
                'min_stock': np.random.randint(50, 200),
                'max_stock': np.random.randint(1000, 10000),
                'unit': 'pcs' if 'Component' in material else 'kg',
                'last_updated': datetime.now() - timedelta(hours=np.random.randint(0, 24))
            }

            # 재고 상태 판정
            if inv['current_stock'] < inv['min_stock']:
                inv['status'] = 'Critical'
            elif inv['current_stock'] < inv['min_stock'] * 1.5:
                inv['status'] = 'Low'
            else:
                inv['status'] = 'Normal'

            inventory.append(inv)

        return pd.DataFrame(inventory)

# MES 시스템 초기화
mes = MESSystem()

print("📋 생산 주문 현황:")
print(mes.production_orders.groupby('status')['order_id'].count())
print("\n📊 품질 검사 결과:")
print(mes.quality_data['result'].value_counts())
print(f"불량률: {(mes.quality_data['result'] == 'Fail').mean()*100:.2f}%")
print("\n📦 재고 상태:")
print(mes.inventory_data.groupby('status')['material'].count())

## 3. 품질/설비/공정 과제 발굴 및 KPI/ROI 정의

### 3.1 문제 정의 프레임워크

In [ ]:
class ProblemDefinitionFramework:
    def __init__(self):
        self.problem_categories = {
            '품질': [
                {'문제': '높은 불량률', 'KPI': 'PPM (Parts Per Million)', 'AI솔루션': '비전검사 딥러닝'},
                {'문제': '품질 편차', 'KPI': 'Cpk (공정능력지수)', 'AI솔루션': 'SPC + 이상탐지'},
                {'문제': '검사 누락', 'KPI': '검출률', 'AI솔루션': '자동 품질 예측 모델'}
            ],
            '설비': [
                {'문제': '예기치 않은 고장', 'KPI': 'MTBF', 'AI솔루션': '예지보전 (진동/온도 분석)'},
                {'문제': '긴 수리 시간', 'KPI': 'MTTR', 'AI솔루션': '고장 원인 자동 진단'},
                {'문제': '낮은 가동률', 'KPI': 'OEE', 'AI솔루션': '최적 운전 조건 추천'}
            ],
            '공정': [
                {'문제': '생산성 저하', 'KPI': 'Throughput', 'AI솔루션': '병목 공정 예측 및 최적화'},
                {'문제': '높은 에너지 소비', 'KPI': 'Energy/Unit', 'AI솔루션': '에너지 소비 최적화'},
                {'문제': '긴 Setup 시간', 'KPI': 'Changeover Time', 'AI솔루션': '최적 스케줄링'}
            ]
        }

    def calculate_roi(self, problem_type, current_value, target_improvement, annual_volume, cost_per_unit):
        """ROI 계산"""
        improvement_rate = target_improvement / 100

        if problem_type == '불량률 감소':
            current_defect_rate = current_value / 100
            new_defect_rate = current_defect_rate * (1 - improvement_rate)
            savings = annual_volume * (current_defect_rate - new_defect_rate) * cost_per_unit

        elif problem_type == 'OEE 향상':
            current_oee = current_value / 100
            new_oee = current_oee * (1 + improvement_rate)
            additional_production = annual_volume * (new_oee - current_oee) / current_oee
            savings = additional_production * cost_per_unit * 0.3  # 이익률 30% 가정

        elif problem_type == 'MTBF 향상':
            current_mtbf = current_value
            new_mtbf = current_mtbf * (1 + improvement_rate)
            downtime_reduction = (8760 / current_mtbf - 8760 / new_mtbf)  # 연간 시간
            savings = downtime_reduction * cost_per_unit  # cost_per_unit은 시간당 손실 비용

        else:
            savings = 0

        # AI 프로젝트 비용 추정 (간단한 모델)
        project_cost = 100000  # 기본 프로젝트 비용 (단위: 천원)

        roi = ((savings - project_cost) / project_cost) * 100
        payback_period = project_cost / (savings / 12) if savings > 0 else float('inf')

        return {
            'annual_savings': savings,
            'project_cost': project_cost,
            'roi_percent': roi,
            'payback_months': payback_period
        }

# 문제 정의 및 ROI 계산 예시
pdf = ProblemDefinitionFramework()

# ROI 계산 예시
roi_cases = [
    {'type': '불량률 감소', 'current': 3, 'improvement': 50, 'volume': 1000000, 'cost': 10},
    {'type': 'OEE 향상', 'current': 65, 'improvement': 15, 'volume': 500000, 'cost': 50},
    {'type': 'MTBF 향상', 'current': 150, 'improvement': 30, 'volume': 0, 'cost': 5000}
]

print("💰 ROI 분석 결과:\n")
for case in roi_cases:
    result = pdf.calculate_roi(case['type'], case['current'], case['improvement'],
                               case['volume'], case['cost'])
    print(f"📌 {case['type']}:")
    print(f"   - 연간 절감액: {result['annual_savings']:,.0f} 천원")
    print(f"   - ROI: {result['roi_percent']:.1f}%")
    print(f"   - 투자 회수 기간: {result['payback_months']:.1f} 개월\n")

### 3.2 KPI Dashboard 구성

In [ ]:
# KPI Dashboard 생성
def create_kpi_dashboard(scada_kpis, mes_data):
    """통합 KPI 대시보드 생성"""

    # KPI 계산
    kpis = scada.calculate_kpis()

    # 대시보드 생성
    fig = go.Figure()

    # OEE Gauge
    fig.add_trace(go.Indicator(
        mode = "gauge+number+delta",
        value = kpis['OEE'] * 100,
        title = {'text': "OEE (%)"},
        delta = {'reference': 85, 'increasing': {'color': "green"}},
        gauge = {
            'axis': {'range': [None, 100]},
            'bar': {'color': "darkblue"},
            'steps': [
                {'range': [0, 50], 'color': "lightgray"},
                {'range': [50, 80], 'color': "gray"}],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 85}},
        domain = {'x': [0, 0.3], 'y': [0.5, 1]}
    ))

    # Quality Rate
    quality_rate = (mes.quality_data['result'] == 'Pass').mean() * 100
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = quality_rate,
        title = {'text': "품질 합격률 (%)"},
        delta = {'reference': 98, 'increasing': {'color': "green"}},
        domain = {'x': [0.35, 0.65], 'y': [0.5, 1]}
    ))

    # MTBF
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = kpis['MTBF'],
        title = {'text': "MTBF (hours)"},
        delta = {'reference': 168, 'increasing': {'color': "green"}},
        domain = {'x': [0.7, 1], 'y': [0.5, 1]}
    ))

    # Throughput
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = kpis['Throughput'],
        title = {'text': "생산량 (units/hour)"},
        delta = {'reference': 100, 'increasing': {'color': "green"}},
        domain = {'x': [0, 0.3], 'y': [0, 0.5]}
    ))

    # Production Orders Status
    order_status = mes.production_orders['status'].value_counts()
    fig.add_trace(go.Pie(
        labels=order_status.index,
        values=order_status.values,
        title='생산 주문 상태',
        domain={'x': [0.35, 0.65], 'y': [0, 0.4]}
    ))

    # Inventory Status
    inv_status = mes.inventory_data['status'].value_counts()
    fig.add_trace(go.Bar(
        x=inv_status.index,
        y=inv_status.values,
        name='재고 상태',
        marker_color=['red' if x=='Critical' else 'yellow' if x=='Low' else 'green' for x in inv_status.index]
    ))

    fig.update_layout(
        title="제조 KPI Dashboard",
        height=600,
        showlegend=False
    )

    return fig

# Dashboard 표시
dashboard = create_kpi_dashboard(scada.kpi_metrics, mes)
dashboard.show()

## 4. 데이터 거버넌스 및 보안

### 4.1 데이터 접근 권한 관리

In [ ]:
class DataGovernance:
    def __init__(self):
        self.access_levels = {
            'Level_0_Operator': {
                'description': '현장 운전자',
                'read': ['PLC', 'HMI'],
                'write': ['HMI'],
                'restricted': ['MES', 'ERP']
            },
            'Level_1_Engineer': {
                'description': '엔지니어',
                'read': ['PLC', 'SCADA', 'MES'],
                'write': ['PLC', 'SCADA'],
                'restricted': ['ERP']
            },
            'Level_2_Manager': {
                'description': '생산 관리자',
                'read': ['PLC', 'SCADA', 'MES', 'ERP'],
                'write': ['MES'],
                'restricted': []
            },
            'Level_3_DataScientist': {
                'description': '데이터 분석가',
                'read': ['Historian', 'MES', 'Quality_DB'],
                'write': ['Analytics_DB'],
                'restricted': ['PLC', 'SCADA']
            }
        }

        self.data_classification = {
            'Public': ['생산량 통계', 'OEE 지표'],
            'Internal': ['품질 데이터', '재고 현황', '설비 상태'],
            'Confidential': ['원가 정보', '고객 정보', '공정 레시피'],
            'Restricted': ['보안 로그', '개인정보', '핵심 기술 데이터']
        }

    def check_access(self, user_level, data_system, operation):
        """접근 권한 확인"""
        if user_level not in self.access_levels:
            return False

        user_permissions = self.access_levels[user_level]

        if data_system in user_permissions['restricted']:
            return False

        if operation == 'read':
            return data_system in user_permissions['read']
        elif operation == 'write':
            return data_system in user_permissions['write']

        return False

    def audit_log(self, user, action, data, timestamp=None):
        """감사 로그 생성"""
        if timestamp is None:
            timestamp = datetime.now()

        log_entry = {
            'timestamp': timestamp,
            'user': user,
            'action': action,
            'data': data,
            'ip_address': '192.168.1.100',  # 시뮬레이션
            'result': 'Success'
        }

        return log_entry

# 데이터 거버넌스 시뮬레이션
dg = DataGovernance()

# 접근 권한 테스트
test_cases = [
    ('Level_0_Operator', 'PLC', 'read'),
    ('Level_0_Operator', 'MES', 'write'),
    ('Level_1_Engineer', 'SCADA', 'write'),
    ('Level_3_DataScientist', 'Historian', 'read'),
    ('Level_3_DataScientist', 'PLC', 'write')
]

print("🔐 접근 권한 테스트 결과:\n")
for user, system, op in test_cases:
    result = dg.check_access(user, system, op)
    icon = "✅" if result else "❌"
    print(f"{icon} {user} - {system} ({op}): {'허용' if result else '거부'}")

# 감사 로그 예시
print("\n📝 감사 로그 샘플:")
log = dg.audit_log('engineer_kim', 'UPDATE', 'PLC_Program_v2.1')
print(json.dumps(log, default=str, indent=2, ensure_ascii=False))

### 4.2 데이터 품질 관리

In [ ]:
class DataQualityManagement:
    def __init__(self, data):
        self.data = data
        self.quality_metrics = {}

    def check_completeness(self):
        """완전성 검사: 결측값 비율"""
        missing_ratio = self.data.isnull().sum() / len(self.data)
        self.quality_metrics['completeness'] = (1 - missing_ratio.mean()) * 100
        return missing_ratio

    def check_consistency(self):
        """일관성 검사: 논리적 오류 검출"""
        inconsistencies = []

        # 예시: 온도가 물리적으로 불가능한 범위
        if 'Temperature_Zone1' in self.data.columns:
            invalid_temp = self.data[(self.data['Temperature_Zone1'] < -273) |
                                    (self.data['Temperature_Zone1'] > 1000)]
            if len(invalid_temp) > 0:
                inconsistencies.append(f"Invalid temperature: {len(invalid_temp)} records")

        self.quality_metrics['consistency'] = (1 - len(inconsistencies)/10) * 100  # 간단한 점수
        return inconsistencies

    def check_timeliness(self):
        """적시성 검사: 데이터 지연 시간"""
        if self.data.index.dtype == 'datetime64[ns]':
            latest_data = self.data.index.max()
            delay = (datetime.now() - pd.to_datetime(latest_data)).total_seconds() / 60
            self.quality_metrics['timeliness'] = max(0, 100 - delay)  # 분 단위 지연
            return delay
        return None

    def check_accuracy(self, reference_data=None):
        """정확성 검사: 참조 데이터와 비교"""
        # 시뮬레이션: 센서 드리프트 감지
        if 'Temperature_Zone1' in self.data.columns:
            mean_temp = self.data['Temperature_Zone1'].mean()
            std_temp = self.data['Temperature_Zone1'].std()

            # 3-sigma 규칙으로 이상값 검출
            outliers = self.data[(self.data['Temperature_Zone1'] < mean_temp - 3*std_temp) |
                                (self.data['Temperature_Zone1'] > mean_temp + 3*std_temp)]

            accuracy_score = (1 - len(outliers)/len(self.data)) * 100
            self.quality_metrics['accuracy'] = accuracy_score
            return len(outliers)
        return 0

    def generate_quality_report(self):
        """종합 데이터 품질 리포트 생성"""
        self.check_completeness()
        self.check_consistency()
        self.check_timeliness()
        self.check_accuracy()

        # 종합 점수 계산
        overall_score = np.mean(list(self.quality_metrics.values()))

        return self.quality_metrics, overall_score

# 데이터 품질 검사 실행
dqm = DataQualityManagement(plc_data)
quality_metrics, overall_score = dqm.generate_quality_report()

print("📊 데이터 품질 평가 결과:\n")
for metric, score in quality_metrics.items():
    status = "🟢" if score >= 90 else "🟡" if score >= 70 else "🔴"
    print(f"{status} {metric.capitalize()}: {score:.2f}%")

print(f"\n종합 데이터 품질 점수: {overall_score:.2f}%")

# 품질 점수 시각화
fig = go.Figure(data=[go.Bar(
    x=list(quality_metrics.keys()),
    y=list(quality_metrics.values()),
    marker_color=['green' if v >= 90 else 'yellow' if v >= 70 else 'red'
                  for v in quality_metrics.values()]
)])

fig.add_hline(y=90, line_dash="dash", line_color="green",
             annotation_text="Good Quality (90%)")
fig.add_hline(y=70, line_dash="dash", line_color="orange",
             annotation_text="Acceptable (70%)")

fig.update_layout(
    title="Data Quality Metrics",
    yaxis_title="Quality Score (%)",
    xaxis_title="Metrics",
    yaxis_range=[0, 100]
)

fig.show()

## 5. 실습 과제

### 과제 1: 자사 제조 프로세스 매핑
- 현재 사용 중인 시스템 (PLC, SCADA, MES, ERP) 식별
- 데이터 흐름도 작성
- 주요 데이터 소스와 형식 정리

### 과제 2: 문제 정의 및 KPI 설정
- Top 3 개선 과제 도출
- 각 과제별 측정 가능한 KPI 정의
- 예상 ROI 계산

### 과제 3: 데이터 거버넌스 계획
- 데이터 접근 권한 매트릭스 작성
- 데이터 품질 체크리스트 작성
- 보안 정책 수립

In [ ]:
# 과제 템플릿 생성
def create_assignment_template():
    template = {
        '과제1_시스템_매핑': {
            'Level_0': [],  # 센서, 액추에이터 목록
            'Level_1': [],  # PLC, DCS 목록
            'Level_2': [],  # SCADA, HMI 목록
            'Level_3': [],  # MES, LIMS 목록
            'Level_4': [],  # ERP, SCM 목록
            '주요_데이터_소스': [],
            '데이터_형식': []  # CSV, JSON, SQL, OPC-UA 등
        },
        '과제2_문제정의': {
            'Problem_1': {
                '문제_설명': '',
                'KPI': '',
                '현재값': 0,
                '목표값': 0,
                'AI_솔루션': '',
                '예상_ROI': 0
            },
            'Problem_2': {},
            'Problem_3': {}
        },
        '과제3_거버넌스': {
            '접근권한': {},
            '품질체크리스트': [],
            '보안정책': []
        }
    }

    # JSON 파일로 저장
    with open('manufacturing_ai_assignment.json', 'w', encoding='utf-8') as f:
        json.dump(template, f, indent=2, ensure_ascii=False)

    print("✅ 과제 템플릿이 'manufacturing_ai_assignment.json' 파일로 생성되었습니다.")
    print("📝 각 섹션을 귀사의 실제 상황에 맞게 작성해주세요.")

    return template

# 템플릿 생성
assignment = create_assignment_template()
print("\n템플릿 구조:")
print(json.dumps(assignment, indent=2, ensure_ascii=False)[:500] + "...")

## 6. 핵심 요약 및 다음 세션 준비

### 이번 세션에서 학습한 내용:
1. **OT/IT 통합 아키텍처**
   - Purdue Model 기반 계층 구조
   - PLC → SCADA → MES → ERP 데이터 흐름
   
2. **문제 정의 및 KPI 설정**
   - 품질/설비/공정별 주요 과제 도출
   - 측정 가능한 KPI 정의 (OEE, MTBF, PPM 등)
   - ROI 계산 방법론
   
3. **데이터 거버넌스**
   - 역할 기반 접근 제어 (RBAC)
   - 데이터 품질 관리 (완전성, 일관성, 적시성, 정확성)
   - 감사 로그 및 보안 정책

### 다음 세션 예고:
**Session 2: 비전 불량검사 고도화 (2-4시간)**
- 실제 불량 이미지 데이터 수집 및 라벨링
- YOLO 모델 학습 및 Fine-tuning
- 세그멘테이션을 통한 정밀 검사
- 오탐/미탐 트레이드오프 최적화

In [ ]:
print("🎯 Session 1 완료!")
print("\n다음 세션을 위한 준비사항:")
print("1. 귀사의 불량 샘플 이미지 준비 (가능한 경우)")
print("2. 주요 불량 유형 리스트 작성")
print("3. 현재 검사 시스템의 성능 지표 (검출률, 오탐률 등) 파악")
print("\n💡 질문이 있으시면 언제든 문의해주세요!")